Data sources:
- [Vehicles from Traficom](https://tieto.traficom.fi/en/datatraficom/open-data?toggle=Open%20data%20for%20vehicles)
- [Municipalities from Statistics Finland](https://stat.fi/en/luokitukset/kunta/)
- [Geographic from MAPOG](https://gisdata.mapog.com/finland/administrative_boundaries_level8_polygon)

In [1]:
# Municipalities data
from src.data_import import get_municipalities

municipalities = get_municipalities()

display(municipalities)

{'020': 'Akaa',
 '005': 'Alajärvi',
 '009': 'Alavieska',
 '010': 'Alavus',
 '016': 'Asikkala',
 '018': 'Askola',
 '019': 'Aura',
 '035': 'Brändö',
 '043': 'Eckerö',
 '046': 'Enonkoski',
 '047': 'Enontekiö',
 '049': 'Espoo',
 '050': 'Eura',
 '051': 'Eurajoki',
 '052': 'Evijärvi',
 '060': 'Finström',
 '061': 'Forssa',
 '062': 'Föglö',
 '065': 'Geta',
 '069': 'Haapajärvi',
 '071': 'Haapavesi',
 '072': 'Hailuoto',
 '074': 'Halsua',
 '075': 'Hamina',
 '076': 'Hammarland',
 '077': 'Hankasalmi',
 '078': 'Hanko',
 '079': 'Harjavalta',
 '081': 'Hartola',
 '082': 'Hattula',
 '086': 'Hausjärvi',
 '111': 'Heinola',
 '090': 'Heinävesi',
 '091': 'Helsinki',
 '097': 'Hirvensalmi',
 '098': 'Hollola',
 '102': 'Huittinen',
 '103': 'Humppila',
 '105': 'Hyrynsalmi',
 '106': 'Hyvinkää',
 '108': 'Hämeenkyrö',
 '109': 'Hämeenlinna',
 '139': 'Ii',
 '140': 'Iisalmi',
 '142': 'Iitti',
 '143': 'Ikaalinen',
 '145': 'Ilmajoki',
 '146': 'Ilomantsi',
 '153': 'Imatra',
 '148': 'Inari',
 '149': 'Ingå',
 '151': 'Isojok

In [2]:
# Vehicles data
from src.data_import import get_vehicles

vehicles = get_vehicles()

display(vehicles.dtypes)
display(vehicles.shape)
display(vehicles.head(10))

registration_date    object
intro_date           object
color                object
driving_force        object
is_hybrid            object
maker_text           object
municipality         object
mileage              object
dtype: object

(2675587, 8)

,registration_date,intro_date,color,driving_force,is_hybrid,maker_text,municipality,mileage
0,02.10.2003,20031002,6,01,NaN,Honda,837,287106
1,17.03.2006,20060317,Y,01,NaN,Toyota,989,161261
2,05.01.2007,20070105,2,02,NaN,Toyota,694,305646
3,14.03.1996,19960314,5,01,NaN,Nissan,777,262907
4,01.07.2003,20030701,8,01,NaN,Honda,851,270594
5,24.03.2000,20000324,6,01,NaN,Nissan,250,189993
6,25.10.2005,20051025,8,01,NaN,BMW,755,218813
7,03.09.2007,20040702,0,02,NaN,Volvo,698,618718
8,21.06.1999,19990621,6,01,NaN,Opel,423,457611
9,16.11.1993,19931116,2,01,NaN,Toyota,091,167672


In [3]:
import copy
import importlib
import src.data_cleaning

importlib.reload(src.data_cleaning)

df = vehicles.copy(deep=True)
mun = copy.deepcopy(municipalities)

df = src.data_cleaning.clean(df, mun)

# Check which high count Other labeled makers are missing from mapping
other_makers = df[df["maker"] == "Other"]
maker_counts = other_makers["maker_text"].value_counts()
top_maker_rows = other_makers[other_makers["maker_text"].isin(maker_counts.index)]
top_maker_counts_in_top_rows = top_maker_rows["maker_text"].value_counts().nlargest(10)
display(top_maker_counts_in_top_rows)

df.reset_index(inplace=True, drop=True)

# NA check
na_rows = df[df[df.columns].isna().any(axis=1)]

# Municipality check
df_mun_codes = set(df["municipality"].unique())
mun_codes = set(mun.keys())
missing = df_mun_codes - mun_codes
display(f"Municipalities mismatch with: {list(missing)}")

# Sanity checks
display(df.dtypes)
display(df.shape)  # Same size as before
display(f"Min year: {df['registration_year'].min()}", f"Max year: {df['registration_year'].max()}")
display(f"Min mileage: {df['mileage'].min()}", f"Max mileage: {df['mileage'].max()}")
display(na_rows)  # Should have none

maker_text
mg           2016
adria        1259
capron       1204
byd          1103
pontiac       677
buick         631
hymer         594
ds            585
gm daewoo     542
dethleffs     536
Name: count, dtype: int64

'Municipalities mismatch with: []'

color                object
driving_force        object
maker_text           object
municipality         object
mileage               Int32
registration_year     Int16
maker                object
dtype: object

(2675587, 7)

'Min year: 1979'

'Max year: 2025'

'Min mileage: -1'

'Max mileage: 5000000'

,color,driving_force,maker_text,municipality,mileage,registration_year,maker


In [4]:
# Final data
import json
import importlib
import os
import src.data_cleaning
import src.data_import
import src.data_validation


importlib.reload(src.data_cleaning)
importlib.reload(src.data_validation)

date = src.data_import.get_date()
final = src.data_cleaning.generate(df, mun, date)
valid = src.data_validation.validate(final, mun)

if valid:
    path = os.path.join(os.path.join(os.getcwd(), "data.json"))
    file = json.dumps(final, indent=2, ensure_ascii=False)
    if os.path.isfile(path):
        os.remove(path)
    with open(path, "w", encoding="utf-8") as fh:
        fh.write(file)

display(final.keys())
display(final["municipalities"][0])

dict_keys(['date', 'municipalities'])

{'code': '020',
 'name': 'Akaa',
 'mileageCount': {'na': 590,
  'under50k': 530,
  '50kto100k': 1184,
  '100kto150k': 1227,
  '150kto200k': 1316,
  '200kto250k': 1415,
  '250kto300k': 1096,
  '300kto350k': 771,
  '350kto400k': 488,
  '400kto450k': 257,
  '450kto500k': 140,
  '500kto550k': 69,
  '550kto600k': 31,
  'over600k': 42},
 'drivingForceCount': {'diesel': 2531,
  'electricity': 368,
  'hybrid': 815,
  'other': 76,
  'petrol': 5366},
 'colorCount': {'black': 1336,
  'blue': 1254,
  'brown': 644,
  'green': 337,
  'grey': 2004,
  'other': 248,
  'red': 1251,
  'silver': 690,
  'white': 1392},
 'registrationYearCount': {'1979': 176,
  '1980': 5,
  '1981': 3,
  '1982': 7,
  '1983': 4,
  '1984': 4,
  '1985': 5,
  '1986': 9,
  '1987': 13,
  '1988': 16,
  '1989': 14,
  '1990': 27,
  '1991': 15,
  '1992': 18,
  '1993': 23,
  '1994': 25,
  '1995': 30,
  '1996': 56,
  '1997': 52,
  '1998': 96,
  '1999': 140,
  '2000': 170,
  '2001': 162,
  '2002': 232,
  '2003': 296,
  '2004': 328,
  '20